<a href="https://colab.research.google.com/github/liangchow/Zindi-Crop/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set Up Worksheet and Import Libraries

In [1]:
# Clone Gitub repository to Colab
from google.colab import drive
drive.mount('/content/drive')

!apt-get install git
!git clone https://github.com/liangchow/Zindi-Crop.git

# Install library
!pip install rasterio

Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Cloning into 'Zindi-Crop'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 25 (delta 8), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (25/25), 2.63 MiB | 11.21 MiB/s, done.
Resolving deltas: 100% (8/8), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 62.2 MB/s eta 0:00:00


In [2]:
# Import libraries
import os
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score

import numpy as np
import rasterio
from rasterio.transform import from_origin

pd.set_option('display.max_columns', None)

# File Preview

In [8]:
# set data dir to Google Drive
DATA_DIR = Path('/content/drive/MyDrive/Zindi-Crop')
os.listdir(DATA_DIR)

['SampleSubmission.csv',
 'Train.csv',
 'StarterNotebook.ipynb',
 'Test.csv',
 'Slides',
 'introduction_to_remote_sensing.ipynb']

In [11]:
# Load files
train = pd.read_csv(DATA_DIR / 'Train.csv')
test = pd.read_csv(DATA_DIR / 'Test.csv')
sample_submission = pd.read_csv(DATA_DIR / 'SampleSubmission.csv')

# Preview file structures
train.shape, test.shape, sample_submission.shape

((7564059, 20), (4946879, 19), (16960, 2))

In [17]:
# Preview training set
train.describe()

,Green,Blue,RED,NIR,SWIR1,SWIR2,Red_Edge,Aerosols,Red_Edge_2,Red_Edge_3,Red_Edge_4,Water_vapor,Cirrus,NDVI,NDMI,NDWI,CI,Target
count,5.214499e+06,5.214028e+06,5.214842e+06,5.214369e+06,5.214003e+06,5.215072e+06,5.214971e+06,5.217058e+06,5.215216e+06,5.215724e+06,5.216182e+06,5.217391e+06,5.213645e+06,5.213989e+06,5.213211e+06,5.214270e+06,5.214509e+06,7.564058e+06
mean,2.684167e-01,2.850368e-01,2.610904e-01,3.875565e-01,2.682408e-01,1.695270e-01,2.705363e-01,3.070151e-01,3.607567e-01,4.173168e-01,4.461057e-01,9.932627e-02,1.949251e-02,3.033574e-01,1.887920e-01,-2.538769e-01,6.334821e-02,8.697515e-01
std,2.075967e-01,2.120771e-01,2.297007e-01,1.746188e-01,1.305292e-01,1.067355e-01,2.140200e-01,2.033707e-01,1.875906e-01,1.865364e-01,1.847119e-01,1.102598e-01,5.539403e-02,2.296761e-01,1.298865e-01,1.718526e-01,8.350355e-02,8.830693e-01
min,4.780000e-02,6.920000e-02,2.740000e-02,5.600000e-02,1.460000e-02,7.000000e-03,2.910000e-02,9.220000e-02,5.590000e-02,6.560000e-02,6.560000e-02,5.700000e-03,3.000000e-04,-5.265151e-01,-5.882127e-01,-7.119470e-01,-3.848435e-01,0.000000e+00
25%,1.110000e-01,1.245000e-01,8.410000e-02,2.632000e-01,1.762000e-01,8.120000e-02,1.099000e-01,1.548000e-01,2.252000e-01,2.817000e-01,3.129000e-01,4.100000e-02,1.300000e-03,8.437500e-02,1.048553e-01,-3.970964e-01,-2.976190e-03,0.000000e+00
50%,1.670000e-01,1.823000e-01,1.515000e-01,3.362000e-01,2.228000e-01,1.337000e-01,1.669000e-01,2.108000e-01,2.903000e-01,3.621000e-01,3.981000e-01,5.850000e-02,2.400000e-03,2.861086e-01,1.858952e-01,-2.349323e-01,3.739837e-02,1.000000e+00
75%,3.865000e-01,4.035000e-01,3.967000e-01,4.763000e-01,3.505000e-01,2.475000e-01,3.927000e-01,4.155000e-01,4.607000e-01,5.161000e-01,5.435000e-01,1.120000e-01,9.900000e-03,5.009613e-01,2.633422e-01,-9.824336e-02,1.142020e-01,2.000000e+00
max,1.296700e+00,1.319500e+00,1.406600e+00,1.352100e+00,8.688000e-01,1.214100e+00,1.344900e+00,1.288500e+00,1.374800e+00,1.479800e+00,1.501400e+00,1.129100e+00,5.925000e-01,8.473346e-01,8.933561e-01,3.919564e-01,5.467024e-01,2.000000e+00


In [15]:
test.describe()

,Green,Blue,RED,NIR,SWIR1,SWIR2,Red_Edge,Aerosols,Red_Edge_2,Red_Edge_3,Red_Edge_4,Water_vapor,Cirrus,NDVI,NDMI,NDWI,CI
count,3.403688e+06,3.403374e+06,3.403744e+06,3.403630e+06,3.403081e+06,3.403665e+06,3.403870e+06,3.405367e+06,3.404036e+06,3.404338e+06,3.404605e+06,3.405521e+06,3.403126e+06,3.403235e+06,3.402572e+06,3.403544e+06,3.403507e+06
mean,2.681880e-01,2.846555e-01,2.607339e-01,3.885132e-01,2.689438e-01,1.698920e-01,2.706381e-01,3.064953e-01,3.614094e-01,4.179822e-01,4.472338e-01,9.973433e-02,1.948364e-02,3.061364e-01,1.891824e-01,-2.561064e-01,6.558362e-02
std,2.077816e-01,2.122416e-01,2.300817e-01,1.743852e-01,1.311437e-01,1.074695e-01,2.141644e-01,2.034609e-01,1.874528e-01,1.863083e-01,1.843848e-01,1.111444e-01,5.655334e-02,2.309471e-01,1.293217e-01,1.725414e-01,8.527499e-02
min,4.910000e-02,7.070000e-02,2.610000e-02,5.590000e-02,1.460000e-02,7.100000e-03,3.020000e-02,9.980000e-02,5.740000e-02,6.620000e-02,6.600000e-02,5.700000e-03,4.000000e-04,-5.265151e-01,-4.726182e-01,-7.088753e-01,-4.085487e-01
25%,1.106000e-01,1.241000e-01,8.350000e-02,2.644000e-01,1.764000e-01,8.100000e-02,1.099000e-01,1.542000e-01,2.259000e-01,2.823000e-01,3.140000e-01,4.160000e-02,1.300000e-03,8.518228e-02,1.060558e-01,-4.010007e-01,-2.684564e-03
50%,1.666000e-01,1.818000e-01,1.508000e-01,3.372000e-01,2.221000e-01,1.328000e-01,1.668000e-01,2.104000e-01,2.910000e-01,3.628000e-01,3.994000e-01,5.950000e-02,2.400000e-03,2.904483e-01,1.859776e-01,-2.383033e-01,3.940274e-02
75%,3.857000e-01,4.027000e-01,3.957000e-01,4.763000e-01,3.522000e-01,2.490000e-01,3.920000e-01,4.148000e-01,4.605000e-01,5.159000e-01,5.440000e-01,1.117000e-01,9.700000e-03,5.056880e-01,2.625162e-01,-9.927291e-02,1.185914e-01
max,1.296700e+00,1.301800e+00,1.406600e+00,1.338600e+00,8.622000e-01,6.868000e-01,1.344900e+00,1.288500e+00,1.374800e+00,1.479800e+00,1.501400e+00,1.129100e+00,5.925000e-01,8.435427e-01,8.933561e-01,2.370892e-01,6.861898e-01


In [16]:
sample_submission.describe()

,Target
count,16960.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0
